In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from math import sqrt

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 0.1
training_epochs = 25
batch_size = 100
display_step = 1
logs_path = './tensorflow_logs/mnist'
# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

In [4]:
x = tf.placeholder(tf.float32, shape=[None, n_input], name='InputData')
y = tf.placeholder(tf.float32, shape=[None, n_classes], name='LabelData')

In [5]:
def NN(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    out_layer = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    return out_layer

In [6]:
# Store layers weight & bias
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1]) / sqrt(n_hidden_1), name='W1'),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]) / sqrt(n_hidden_2), name='W2'),
    'w3': tf.Variable(tf.random_normal([n_hidden_2, n_classes]) / sqrt(n_classes), name='W3')
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]) / sqrt(n_hidden_1), name='b1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]) / sqrt(n_hidden_2), name='b2'),
    'b3': tf.Variable(tf.random_normal([n_classes]) / sqrt(n_classes), name='b3')
}

In [7]:
with tf.name_scope('Model'):
    # Model
    pred = NN(x, weights, biases)
with tf.name_scope('Cost'):
    # Minimize error using cross entropy
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [8]:
# Initializing the variables
init = tf.initialize_all_variables()

# Create a summary to monitor cost tensor
tf.scalar_summary("cost", cost)
# Create a summary to monitor accuracy tensor
tf.scalar_summary("accuracy", accuracy)

# Merge all summaries into a single op
merged_summary_op = tf.merge_all_summaries()

In [9]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer = tf.train.SummaryWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in xrange(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in xrange(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, acc, summary = sess.run([optimizer, cost, accuracy, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost =", "{:.9f}".format(avg_cost), "acc =", "{:.9f}".format(acc)

    print "Optimization Finished!"

    # Test model
    # Calculate accuracy
    print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})

    print "Run the command line:\n" \
          "--> tensorboard --logdir=./tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser"

Epoch: 0001 cost = 0.289970042 acc = 0.959999979
Epoch: 0002 cost = 0.123027408 acc = 0.959999979
Epoch: 0003 cost = 0.085527643 acc = 0.980000019
Epoch: 0004 cost = 0.064776086 acc = 0.970000029
Epoch: 0005 cost = 0.051037293 acc = 0.990000010
Epoch: 0006 cost = 0.040288309 acc = 0.970000029
Epoch: 0007 cost = 0.032156224 acc = 0.980000019
Epoch: 0008 cost = 0.025926097 acc = 1.000000000
Epoch: 0009 cost = 0.020858079 acc = 1.000000000
Epoch: 0010 cost = 0.017063267 acc = 1.000000000
Epoch: 0011 cost = 0.013928600 acc = 1.000000000
Epoch: 0012 cost = 0.011349863 acc = 1.000000000
Epoch: 0013 cost = 0.009299547 acc = 1.000000000
Epoch: 0014 cost = 0.008013693 acc = 1.000000000
Epoch: 0015 cost = 0.006608857 acc = 0.990000010
Epoch: 0016 cost = 0.005731012 acc = 1.000000000
Epoch: 0017 cost = 0.004903970 acc = 1.000000000
Epoch: 0018 cost = 0.004317529 acc = 1.000000000
Epoch: 0019 cost = 0.003811559 acc = 1.000000000
Epoch: 0020 cost = 0.003429478 acc = 1.000000000
Epoch: 0021 cost = 0

In [10]:
print mnist.train.num_examples

55000
